<a href="https://colab.research.google.com/github/Ejedavy/OOP_NN/blob/main/Basic_Mnist_WIth_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
def downloadMnistDatasets():
    trainData = datasets.MNIST(
        root = "data",
        download = True,
        train = True,
        transform = ToTensor()
    )
    
    validationData = datasets.MNIST(
        root = "data",
        download = True,
        train = False,
        transform = ToTensor()
    )
    
    return trainData , validationData

In [3]:
trainData , validatitonData = downloadMnistDatasets()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
BATCH_SIZE = 128

CREATING THE DATALOADER FOR THE TRAINSET

In [5]:
dataloader = DataLoader(trainData, batch_size  = BATCH_SIZE)

In [6]:
class FeedForwardNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.denseLayers = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256,10),
        )
        self.softmax = nn.Softmax(dim = 1)
        
    def forward(self, inputData):
        x = self.flatten(inputData)
        logits = self.denseLayers(x)
        prediction = self.softmax(logits)
        return prediction

In [7]:
if torch.cuda.is_available:
    device = "cuda"
else:
    device = "cpu"
    
print(device)

cuda


In [8]:
model = FeedForwardNet().to(device)

In [9]:
def trainOneEpoch(model, dataloader, lossFn, optimiser, device):
    for inputs, target in dataloader:
        inputs,target = inputs.to(device) , target.to(device)
        
        #Calculating the loss of the model
        prectictions = model(inputs)
        loss = lossFn(prectictions, target)
        
        #BackPropagation loss
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
    print(f"The loss is {loss.item()}") 

In [10]:
def train(model, dataloader, lossFn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i +1}")
        trainOneEpoch(model, dataloader, lossFn, optimiser, device)
        print("------------------------------------")
    print("Training is complete")

In [11]:
EPOCHS = 20
lossFn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(),lr = .001)

Training and saving the model

In [12]:
train(model = model, dataloader=dataloader, lossFn=lossFn,optimiser=optimiser,device = device,epochs=EPOCHS)
torch.save(model.state_dict(),"mnist.pth")

Epoch 1
The loss is 1.5075441598892212
------------------------------------
Epoch 2
The loss is 1.4935272932052612
------------------------------------
Epoch 3
The loss is 1.4853310585021973
------------------------------------
Epoch 4
The loss is 1.4788180589675903
------------------------------------
Epoch 5
The loss is 1.475661277770996
------------------------------------
Epoch 6
The loss is 1.4733489751815796
------------------------------------
Epoch 7
The loss is 1.4725059270858765
------------------------------------
Epoch 8
The loss is 1.4731745719909668
------------------------------------
Epoch 9
The loss is 1.4735159873962402
------------------------------------
Epoch 10
The loss is 1.4736419916152954
------------------------------------
Epoch 11
The loss is 1.4727863073349
------------------------------------
Epoch 12
The loss is 1.4733690023422241
------------------------------------
Epoch 13
The loss is 1.4752178192138672
------------------------------------
Epoch 14
The